## law_1.docx, law_2.docx 두 개 파일 모두 읽기  
- pinecone  store 저장
    - index name: 임의 
- RetrievalQA 구현 
    - prompt: rlm/rag-prompt
    - 질문: 전세사기 관련 질문

In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 임베딩 -> 벡터 스토어(데이터베이스)에 저장
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'quiz04-law1-2'

## 저장된 인덱스 가져오기
# [방법 2]
database = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding=embedding,
)

## RetrievaQA
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": database.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("전세사기피해주택이란 무엇인가요?")
# qa_chain.invoke("전세사기피해자 임대인을 알려주세요")

c:\Users\1\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'전세사기피해주택이란 전세사기피해자가 임차인인 임대차계약의 목적물인 주택을 의미합니다. 이는 주택임대차보호법에 따라 임대차가 끝난 후 임차권등기가 마친 주택도 포함됩니다. 이러한 주택은 전세사기로 인한 피해를 입은 임차인을 보호하기 위한 특별법에 의해 정의됩니다.'